# Module 4.3: Softmax Regression - 多分類

## 學習目標

完成這個 notebook 後，你將能夠：

1. 理解 Softmax 函數的數學定義與性質
2. 理解 Cross-Entropy Loss 用於多分類
3. 推導並實作 Softmax Regression 的梯度
4. 處理多分類問題

## 背景知識

Softmax Regression 是 Logistic Regression 的多分類推廣。將 K 個線性分數轉換成 K 個機率。

---

## 參考資源

- Bishop PRML Ch. 4.3: Probabilistic Discriminative Models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("環境設定完成！")

---

## Part 1: Softmax 函數

### 定義

給定 K 個分數 $z = [z_1, z_2, ..., z_K]$，Softmax 輸出 K 個機率：

$$\text{softmax}(z)_k = \frac{e^{z_k}}{\sum_{j=1}^K e^{z_j}}$$

### 性質

1. 輸出和為 1：$\sum_k \text{softmax}(z)_k = 1$
2. 所有輸出都是正的
3. 放大差異：最大值對應的機率會很大

### 數值穩定性

$e^{z_k}$ 可能溢出。解法：減去最大值

$$\text{softmax}(z)_k = \frac{e^{z_k - \max(z)}}{\sum_j e^{z_j - \max(z)}}$$

In [ ]:
def softmax(z):
    """
    數值穩定的 Softmax 函數
    
    Parameters
    ----------
    z : np.ndarray, shape (N, K) or (K,)
        輸入分數
    
    Returns
    -------
    np.ndarray
        機率分布，和輸入形狀相同
    """
    z = np.atleast_2d(z)
    
    # 減去最大值（數值穩定性）
    z_shifted = z - np.max(z, axis=1, keepdims=True)
    
    # 計算 exp
    exp_z = np.exp(z_shifted)
    
    # 正規化
    probs = exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    return probs.squeeze()


# 測試
print("=== 測試 Softmax ===")

z_test = np.array([2.0, 1.0, 0.1])
probs = softmax(z_test)

print(f"輸入分數: {z_test}")
print(f"Softmax 輸出: {probs}")
print(f"輸出和: {probs.sum():.6f} (應該是 1)")

# 數值穩定性測試
z_large = np.array([1000, 1001, 1002])
probs_large = softmax(z_large)
print(f"\n大數值測試: {z_large}")
print(f"Softmax 輸出: {probs_large} (不應該有 nan 或 inf)")

In [ ]:
# 視覺化 Softmax 的「放大差異」效果

# 固定兩個分數，改變第三個
z1, z2 = 1.0, 0.0
z3_range = np.linspace(-5, 5, 100)

probs = np.array([softmax([z1, z2, z3]) for z3 in z3_range])

plt.figure(figsize=(10, 6))
plt.plot(z3_range, probs[:, 0], 'r-', label='P(class 0) [z=1]')
plt.plot(z3_range, probs[:, 1], 'g-', label='P(class 1) [z=0]')
plt.plot(z3_range, probs[:, 2], 'b-', label='P(class 2) [z=z3]')
plt.xlabel('$z_3$')
plt.ylabel('Probability')
plt.title('Softmax: 改變 $z_3$ 對各類機率的影響')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 2: Cross-Entropy Loss（多分類）

### One-Hot Encoding

標籤 $y = k$ 轉換為 one-hot 向量：

$$y_{\text{one-hot}} = [0, ..., 0, 1, 0, ..., 0]$$ （第 k 位是 1）

### Cross-Entropy Loss

$$L = -\sum_{k=1}^K y_k \log(\hat{y}_k)$$

由於 $y$ 是 one-hot，只有一項非零：

$$L = -\log(\hat{y}_{\text{true class}})$$

In [ ]:
def one_hot_encode(y, n_classes):
    """
    將標籤轉換為 one-hot 編碼
    
    Parameters
    ----------
    y : np.ndarray, shape (N,)
        類別標籤（整數 0 到 K-1）
    n_classes : int
        類別數 K
    
    Returns
    -------
    np.ndarray, shape (N, K)
        One-hot 編碼
    """
    N = len(y)
    one_hot = np.zeros((N, n_classes))
    one_hot[np.arange(N), y.astype(int)] = 1
    return one_hot


def cross_entropy_loss(y_true, y_pred, eps=1e-15):
    """
    計算 Cross-Entropy Loss（多分類）
    
    Parameters
    ----------
    y_true : np.ndarray, shape (N,) or (N, K)
        真實標籤（整數或 one-hot）
    y_pred : np.ndarray, shape (N, K)
        預測機率
    eps : float
        避免 log(0)
    
    Returns
    -------
    float
        平均 Cross-Entropy Loss
    """
    y_pred = np.clip(y_pred, eps, 1 - eps)
    
    if y_true.ndim == 1:
        # 標籤是整數，直接索引
        N = len(y_true)
        return -np.mean(np.log(y_pred[np.arange(N), y_true.astype(int)]))
    else:
        # 標籤是 one-hot
        return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))


# 測試
print("=== 測試 Cross-Entropy Loss ===")

y_true = np.array([0, 1, 2])
y_pred = np.array([[0.7, 0.2, 0.1],
                   [0.1, 0.8, 0.1],
                   [0.2, 0.2, 0.6]])

loss = cross_entropy_loss(y_true, y_pred)
print(f"真實標籤: {y_true}")
print(f"預測機率:\n{y_pred}")
print(f"Cross-Entropy Loss: {loss:.4f}")

# 驗證：手動計算
manual_loss = -np.mean([np.log(0.7), np.log(0.8), np.log(0.6)])
print(f"手動計算: {manual_loss:.4f}")

---

## Part 3: 梯度推導

### 模型

$$z = XW + b$$
$$\hat{y} = \text{softmax}(z)$$

其中 $W \in \mathbb{R}^{D \times K}$，$b \in \mathbb{R}^K$

### 漂亮的結果

$$\frac{\partial L}{\partial z} = \hat{y} - y$$

這和 Logistic Regression 一樣！

因此：

$$\frac{\partial L}{\partial W} = \frac{1}{N} X^T (\hat{y} - y)$$
$$\frac{\partial L}{\partial b} = \frac{1}{N} \sum_i (\hat{y}_i - y_i)$$

In [ ]:
def compute_softmax_gradients(X, y, W, b):
    """
    計算 Softmax Regression 的梯度
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
    y : np.ndarray, shape (N,)
        類別標籤（整數）
    W : np.ndarray, shape (D, K)
    b : np.ndarray, shape (K,)
    
    Returns
    -------
    dW : np.ndarray, shape (D, K)
    db : np.ndarray, shape (K,)
    """
    N = X.shape[0]
    K = W.shape[1]
    
    # 前向傳播
    z = X @ W + b
    y_pred = softmax(z)  # shape (N, K)
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(1, -1)
    
    # One-hot 編碼
    y_one_hot = one_hot_encode(y, K)  # shape (N, K)
    
    # 梯度
    error = y_pred - y_one_hot  # shape (N, K)
    
    dW = (1 / N) * (X.T @ error)  # shape (D, K)
    db = (1 / N) * np.sum(error, axis=0)  # shape (K,)
    
    return dW, db


# 數值梯度檢查
def numerical_gradient_matrix(f, x, eps=1e-5):
    """計算矩陣的數值梯度"""
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        old_value = x[idx]
        
        x[idx] = old_value + eps
        fx_plus = f(x)
        
        x[idx] = old_value - eps
        fx_minus = f(x)
        
        x[idx] = old_value
        grad[idx] = (fx_plus - fx_minus) / (2 * eps)
        
        it.iternext()
    
    return grad


def gradient_check_softmax(X, y, W, b, eps=1e-5):
    """檢查 Softmax 梯度"""
    K = W.shape[1]
    
    # 解析梯度
    dW_analytic, db_analytic = compute_softmax_gradients(X, y, W, b)
    
    # 數值梯度 for W
    def loss_W(W_test):
        z = X @ W_test + b
        y_pred = softmax(z)
        if y_pred.ndim == 1:
            y_pred = y_pred.reshape(1, -1)
        return cross_entropy_loss(y, y_pred)
    
    dW_numeric = numerical_gradient_matrix(loss_W, W.copy(), eps)
    
    # 數值梯度 for b
    def loss_b(b_test):
        z = X @ W + b_test
        y_pred = softmax(z)
        if y_pred.ndim == 1:
            y_pred = y_pred.reshape(1, -1)
        return cross_entropy_loss(y, y_pred)
    
    db_numeric = numerical_gradient_matrix(loss_b, b.copy(), eps)
    
    # 比較
    print("=== 梯度檢查 ===")
    rel_err_W = np.linalg.norm(dW_analytic - dW_numeric) / (np.linalg.norm(dW_analytic) + np.linalg.norm(dW_numeric) + 1e-8)
    rel_err_b = np.linalg.norm(db_analytic - db_numeric) / (np.linalg.norm(db_analytic) + np.linalg.norm(db_numeric) + 1e-8)
    
    print(f"dW 相對誤差: {rel_err_W:.2e}")
    print(f"db 相對誤差: {rel_err_b:.2e}")
    
    passed = rel_err_W < 1e-4 and rel_err_b < 1e-4
    print(f"梯度檢查通過: {passed}")
    return passed


# 測試
np.random.seed(42)
X_test = np.random.randn(20, 5)
y_test = np.random.randint(0, 3, 20)
W_test = np.random.randn(5, 3) * 0.1
b_test = np.random.randn(3) * 0.1

gradient_check_softmax(X_test, y_test, W_test, b_test)

---

## Part 4: 完整 Softmax Regression 實作

In [ ]:
class SoftmaxRegression:
    """
    Softmax Regression 多分類模型
    
    Parameters
    ----------
    n_classes : int
        類別數
    regularization : float
        L2 正則化強度
    """
    
    def __init__(self, n_classes, regularization=0.0):
        self.n_classes = n_classes
        self.reg = regularization
        self.W = None
        self.b = None
        self.history = None
    
    def _softmax(self, z):
        """數值穩定的 Softmax"""
        z = np.atleast_2d(z)
        z_shifted = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z_shifted)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def fit(self, X, y, lr=0.1, n_iter=1000, verbose=True):
        """
        訓練模型
        
        Parameters
        ----------
        X : np.ndarray, shape (N, D)
        y : np.ndarray, shape (N,)
            類別標籤（整數 0 到 K-1）
        lr : float
            學習率
        n_iter : int
            迭代次數
        """
        N, D = X.shape
        K = self.n_classes
        
        # 初始化
        self.W = np.random.randn(D, K) * 0.01
        self.b = np.zeros(K)
        self.history = {'loss': [], 'accuracy': []}
        
        # One-hot 編碼
        y_one_hot = one_hot_encode(y, K)
        
        for i in range(n_iter):
            # 前向傳播
            z = X @ self.W + self.b
            y_pred = self._softmax(z)
            
            # Loss
            loss = cross_entropy_loss(y, y_pred) + 0.5 * self.reg * np.sum(self.W ** 2)
            
            # Accuracy
            predictions = np.argmax(y_pred, axis=1)
            accuracy = np.mean(predictions == y)
            
            self.history['loss'].append(loss)
            self.history['accuracy'].append(accuracy)
            
            # 梯度
            error = y_pred - y_one_hot
            dW = (1 / N) * (X.T @ error) + self.reg * self.W
            db = (1 / N) * np.sum(error, axis=0)
            
            # 更新
            self.W = self.W - lr * dW
            self.b = self.b - lr * db
            
            if verbose and (i + 1) % (n_iter // 10) == 0:
                print(f"Iter {i+1:4d}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
        
        return self
    
    def predict_proba(self, X):
        """預測機率"""
        z = X @ self.W + self.b
        return self._softmax(z)
    
    def predict(self, X):
        """預測類別"""
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)
    
    def score(self, X, y):
        """計算準確率"""
        predictions = self.predict(X)
        return np.mean(predictions == y)


# 生成多分類資料
def generate_multiclass_data(n_samples=300, n_classes=3, seed=42):
    """
    生成多分類測試資料
    """
    np.random.seed(seed)
    
    n_per_class = n_samples // n_classes
    
    X_list = []
    y_list = []
    
    # 每個類別的中心（等間隔分布在圓上）
    angles = np.linspace(0, 2 * np.pi, n_classes, endpoint=False)
    radius = 2.0
    
    for k in range(n_classes):
        center = np.array([radius * np.cos(angles[k]), radius * np.sin(angles[k])])
        X_k = np.random.randn(n_per_class, 2) * 0.7 + center
        X_list.append(X_k)
        y_list.append(np.full(n_per_class, k))
    
    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    
    idx = np.random.permutation(len(y))
    return X[idx], y[idx]


# 測試
print("=== 測試 Softmax Regression ===")

X, y = generate_multiclass_data(n_samples=300, n_classes=3)

print(f"資料大小: {X.shape}")
print(f"類別分布: {np.bincount(y.astype(int))}")

# 訓練
model = SoftmaxRegression(n_classes=3, regularization=0.01)
model.fit(X, y, lr=0.5, n_iter=500, verbose=True)

print(f"\n最終準確率: {model.score(X, y):.4f}")

In [ ]:
# 視覺化

def plot_multiclass_decision_boundary(model, X, y):
    """繪製多分類決策邊界"""
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='viridis')
    plt.contour(xx, yy, Z, colors='black', linewidths=0.5)
    
    colors = ['blue', 'red', 'green', 'purple', 'orange']
    markers = ['o', 's', '^', 'D', 'v']
    
    for k in range(model.n_classes):
        mask = y == k
        plt.scatter(X[mask, 0], X[mask, 1], c=colors[k % len(colors)], 
                   marker=markers[k % len(markers)], s=50, 
                   label=f'Class {k}', edgecolors='k')
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title('Softmax Regression 決策邊界')
    plt.legend()
    plt.grid(True, alpha=0.3)


plot_multiclass_decision_boundary(model, X, y)
plt.show()

# 訓練曲線
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(model.history['loss'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss')
axes[0].set_title('訓練 Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(model.history['accuracy'])
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('訓練準確率')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 練習題

### 練習 1：更多類別

In [ ]:
# 練習 1 解答：5 類別分類

print("=== 5 類別分類測試 ===")

X5, y5 = generate_multiclass_data(n_samples=500, n_classes=5)

model5 = SoftmaxRegression(n_classes=5, regularization=0.01)
model5.fit(X5, y5, lr=0.3, n_iter=1000, verbose=True)

print(f"\n最終準確率: {model5.score(X5, y5):.4f}")

plot_multiclass_decision_boundary(model5, X5, y5)
plt.title('5 類別 Softmax Regression')
plt.show()

### 練習 2：混淆矩陣

In [ ]:
# 練習 2 解答

def compute_confusion_matrix(y_true, y_pred, n_classes):
    """
    計算混淆矩陣
    
    Returns
    -------
    np.ndarray, shape (n_classes, n_classes)
        confusion[i, j] = 真實類別 i 被預測為 j 的次數
    """
    confusion = np.zeros((n_classes, n_classes), dtype=int)
    
    for true, pred in zip(y_true.astype(int), y_pred.astype(int)):
        confusion[true, pred] += 1
    
    return confusion


def plot_confusion_matrix(confusion, class_names=None):
    """視覺化混淆矩陣"""
    n_classes = confusion.shape[0]
    
    if class_names is None:
        class_names = [f'Class {i}' for i in range(n_classes)]
    
    plt.figure(figsize=(8, 6))
    plt.imshow(confusion, cmap='Blues')
    plt.colorbar(label='Count')
    
    # 標籤
    plt.xticks(range(n_classes), class_names)
    plt.yticks(range(n_classes), class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    
    # 數值
    for i in range(n_classes):
        for j in range(n_classes):
            plt.text(j, i, str(confusion[i, j]), 
                    ha='center', va='center', fontsize=12,
                    color='white' if confusion[i, j] > confusion.max() / 2 else 'black')


# 計算混淆矩陣
y_pred = model.predict(X)
confusion = compute_confusion_matrix(y, y_pred, 3)

print("混淆矩陣:")
print(confusion)

plot_confusion_matrix(confusion)
plt.show()

# 計算每類準確率
print("\n每類準確率:")
for i in range(3):
    class_acc = confusion[i, i] / np.sum(confusion[i, :])
    print(f"  Class {i}: {class_acc:.4f}")

---

## 總結

### 本 Notebook 涵蓋的內容

1. **Softmax 函數**：
   - 將 K 個分數轉為 K 個機率
   - 數值穩定：減去最大值

2. **Cross-Entropy Loss**：
   - $L = -\log(\hat{y}_{\text{true class}})$
   - One-hot 編碼

3. **梯度**：
   - $\frac{\partial L}{\partial z} = \hat{y} - y$
   - 和 Logistic Regression 形式相同

4. **決策邊界**：
   - 線性邊界
   - K 類會有 K 個區域

### 關鍵要點

1. Softmax = Sigmoid 的多分類推廣
2. 數值穩定性很重要
3. 梯度形式統一簡潔

### 下一步

在下一個 notebook 中，我們將學習 **SVM（支持向量機）**。